# Geospatial Analysis of the Flood Impacts on Landcover and Buildings in Maiduguri Nigeria for September 2024

#### Data Acquisition:
- We start by acquiring Sentinel-1 data for September 2024 over Maiduguri. The script uses pystac to search for the necessary data via a STAC Catalog (e.g., the Data Space Browser) and Google Earth Engine.

- The code then downloads OpenStreetMap data for Maiduguri, including buildings and land use information, using the overpy library to query the OpenStreetMap API.

#### Data Processing:
- Sentinel-1 data is processed using rasterio to open the downloaded image and extract relevant bands.
We then create an xarray Dataset for easier analysis and create a water mask using NDWI.

- The NDVI is calculated, and a simple threshold is applied.

- OpenStreetMap data is processed to extract features (buildings and land use). The code then uses geopandas to create GeoDataFrames and reproject them to the same CRS as the Sentinel-1 data.

#### Analysis and Visualization:
- The code combines the Sentinel-1 and OpenStreetMap data.
- It calculates the flooded cropland area by intersecting the water mask with the cropland layer.
- It then visualizes the data using Matplotlib. The code creates a flood extent map, a crop impact map showing NDVI, and a vulnerability map using the flooded cropland ratio.

#### 3D Visualization: 
The code creates a PyVista grid to visualize the NDVI data in 3D using pyvista.


#### Output and Communication
... (Generate a report, share data, and communicate findings) ...


In [4]:
# Import necessary libraries
import os
import datetime
import time
import requests
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ee
import geemap
import osmnx as ox
import shapely
from shapely.geometry import Polygon, mapping, LineString, shape
import shapely.geometry as sg
from shapely.geometry.polygon import orient
from shapely.ops import transform
import rasterio
import rioxarray
import xarray as xr
import zarr
import pystac
from pystac import Catalog
from pystac.extensions.projection import ProjectionExtension
import dask


In [5]:
# Better visualization
plt.style.use('ggplot')

In [6]:
DATA_DIR = 'maiduguri_flood_archive'
ZARR_STORE = os.path.join(DATA_DIR, 'maiduguri.zarr')

In [ ]:
# client = Client('127.0.0.1:8787')

In [5]:
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
else :
    pass
    # shutil.rmtree(DATA_DIR)
    # os.makedirs(DATA_DIR)

In [6]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./log/log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [7]:
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', "clear-router-390022")


In [6]:
geemap.ee_initialize(project=EE_PROJECT_ID)

TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000187A1678100>: Failed to resolve 'oauth2.googleapis.com' ([Errno 11001] getaddrinfo failed)"))

In [9]:
geojson_data = {"type":"Polygon","coordinates":[[[13.059998,11.836439],[13.318176,12.136005],[13.454132,12.035291],[13.242645,11.790736],[13.271484,11.757126],[13.233032,11.734269],[13.059998,11.836439]]]}
# 810.87 km2
# Extract the coordinates
coordinates = geojson_data["coordinates"][0]

# Find the minimum and maximum longitude and latitude
min_lon = min([coord[0] for coord in coordinates])
max_lon = max([coord[0] for coord in coordinates])
min_lat = min([coord[1] for coord in coordinates])
max_lat = max([coord[1] for coord in coordinates])

# Construct the bounding box
bbox = [min_lon, min_lat, max_lon, max_lat]

print("bbox:", bbox)


bbox: [13.059998, 11.734269, 13.454132, 12.136005]


In [10]:
# Using ee.Geometry.Polygon
roi = ee.Geometry.Polygon(coordinates)
display(roi)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              13.059998,
              11.836439
            ],
            [
              13.318176,
              12.136005
            ],
            [
              13.454132,
              12.035291
            ],
            [
              13.242645,
              11.790736
            ],
            [
              13.271484,
              11.757126
            ],
            [
              13.233032,
              11.734269
            ],
            [
              13.059998,
              11.836439
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [11]:
# Extract center coordinates
bbox_coords = geojson_data["coordinates"][0]
longitude = sum([coord[0] for coord in bbox_coords]) / len(bbox_coords)
latitude = sum([coord[1] for coord in bbox_coords]) / len(bbox_coords)
center = [latitude, longitude] 
display(center)

[11.875186428571428, 13.234209285714286]

In [ ]:
# ## 1. Data Acquisition

# **1.1 Download Sentinel-2 Data**

# Create the STAC Catalog

catalog = pystac.Catalog(
    id="kimongo-catalog",
    title="Kimongo Sentinel-2 Catalog",
    description="STAC Catalog for Sentinel-2 Baseline Product in the Kimongo Area.",
)


In [ ]:
# Function to parse ISO 8601 date strings with 'Z'
def parse_iso_date(iso_date):
    return datetime.strptime(iso_date.replace("Z", "+00:00"), "%Y-%m-%dT%H:%M:%S.%f%z")

# Create a STAC Item for the product
baseline_stac_item = pystac.Item(
    id="S2A_MSIL2A_20181117T091241_N0213_R050_T33MTR_20201129T230722.SAFE",
    geometry={
        "type": "Polygon",
        "coordinates": [
            [   
                [13.059998, 11.836439],
                [13.233032, 11.734269],
                [13.271484, 11.757126],
                [13.242645, 11.790736],
                [13.454132, 12.035291],
                [13.318176, 12.136005],
                [13.059998, 11.836439]
            ]
        ]
    },
    bbox=(13.059998, 11.734269, 13.454132, 12.136005),
    datetime=parse_iso_date("2018-11-17T09:12:41.024000Z"),
    properties={
        "platform": "SENTINEL-2",
        "cloud_coverage": 15.5165,
        "beginning_datetime": "2018-11-17T09:12:41.024000Z",
        "ending_datetime": "2018-11-17T09:12:41.024000Z",
        "absolute_orbit": 17779,
        "instrument": "MSI",
        "acquisition_mode": "INS-NOBS",
        "processing_level": "S2MSI2A",
        "tile_id": "33MTR",
        "spatial_resolution": 60.0,
        "published": "2020-11-29T23:50:08.138215Z",
        "size": "< 1MB",
        "product_id": "https://zipper.dataspace.copernicus.eu/odata/v1/Products(2f9d2507-208f-5e71-a973-18f02458d48f)/$value",
        "download_link": "https://link.dataspace.copernicus.eu/yxsh"
    }
)

# Use S3 path as the asset href
baseline_stac_item.add_asset(
        key="data",
        asset=pystac.Asset(
            href="/eodata/Sentinel-2/MSI/L2A/2018/11/17/S2A_MSIL2A_20181117T091241_N0213_R050_T33MTR_20201129T230722.SAFE",
            media_type=pystac.MediaType.GEOTIFF,
            title="Sentinel-2 S3 Data"
        )
    )

baseline_stac_item.add_asset(
    key="BaseB04",
    asset=pystac.Asset(
        href="./Browser_images/SENTINEL-2/Baseline/2018-07-31-00_00_2023-07-30-23_59_Sentinel-2_L2A_B04_(Raw).tiff",
        media_type=pystac.MediaType.TIFF,
        title="Sentinel-2 Baseline B04"
    )
)

baseline_stac_item.add_asset(
    key="BaseB08",
    asset=pystac.Asset(
        href="./Browser_images/SENTINEL-2/Baseline/2018-07-31-00_00_2023-07-30-23_59_Sentinel-2_L2A_B08_(Raw).tiff",
        media_type=pystac.MediaType.TIFF,
        title="Sentinel-2 Baseline B08"
    )
)



In [ ]:
# Add the item to the catalog
catalog.add_item(baseline_stac_item)

# Create a directory for the STAC catalog if it doesn't exist
output_directory = './stac_catalog'
os.makedirs(output_directory, exist_ok=True)

# Get the current date and time
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create the output file path with datetime
output_file_path = os.path.join(output_directory, f'baseline_catalog-{current_datetime}')

# Normalize hrefs and save the STAC catalog to a file
catalog.normalize_hrefs(output_directory)
catalog.save(catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED, dest_href=output_file_path)

print(f"STAC catalog saved to {output_file_path}")

print("STAC Item created:")

display(baseline_stac_item.to_dict())

log_progress('Baseline stac creation complete. Initiating process_baseline_analysis')


In [17]:
# Load Sentinel-2 image collection
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [18]:
# Filter the collection for time range for image during the flood
start_date = '2024-08-14'
end_date = '2024-09-14'
s2_Image_flood = s2_collection.filterDate(start_date, end_date).filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 50).filterBounds(roi)

In [19]:
s2_Image_flood = s2_Image_flood.mosaic().clip(roi)

In [20]:
# Create an interactive map
Map1 = geemap.Map(center=center, zoom = 12)
Map1.add_basemap("ROADMAP")

In [21]:
# Visualize the image during the flood
s2_Viz = {"min":0.0, "max":3000, "bands":["B4", "B3", "B2"]}
Map1.addLayer(s2_Image_flood, s2_Viz, "s2_Image_flood")
Map1
#Map.centerObject(s2_Image_flood, 8)

Map(center=[11.875186428571428, 13.234209285714286], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
tile_id = s2_collection.get('MGRS_TILE').getInfo()

In [23]:
# Load Pre_flood images
 
start_date = '2024-06-01'
end_date = '2024-06-30'
s2_Collection_pre = s2_collection.filterDate(start_date, end_date).filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 50).filterBounds(roi).getInfo()

display(s2_Collection_pre)

{'type': 'ImageCollection',
 'bands': [],
 'version': 1727605278145572,
 'id': 'COPERNICUS/S2_SR_HARMONIZED',
 'properties': {'date_range': [1490659200000, 1647907200000],
  'period': 0,
  'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'keywords': ['copernicus',
   'esa',
   'eu',
   'msi',
   'reflectance',
   'sentinel',
   'sr'],
  'system:visualization_0_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/COPERNICUS_S2_SR_thumb.png',
  'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 L2 data are downloaded from scihub. They were\ncomputed by running sen2cor. WARNING: ESA did not produce L2 data\nfor all L1 assets, and earlier L2 coverage is not global.</p><p>The assets contain\n12 UINT16 spectral bands 

In [30]:
# tiles = ["33PUN","33PUP","33PTN",]
# .filter(ee.Filter.inList("MGRS_TILE", tiles))

s2_Collection_pre = s2_collection.filterDate(start_date, end_date).filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 50).filterBounds(roi)


In [31]:
# Check the numbers of image in the collection
print("The number of available images before flood is:", s2_Collection_pre.size().getInfo())

The number of available images before flood is: 8


In [32]:
# Create a mosaic from the s2_Collection_Pre

s2_Image_pre = s2_Collection_pre.mosaic().clip(roi)

In [33]:
# Visualize the image before the flood
Map2 = geemap.Map(center=center, zoom = 12)
Map2.addLayer(s2_Image_pre, s2_Viz, "s2_Image_pre")
Map2

Map(center=[11.875186428571428, 13.234209285714286], controls=(WidgetControl(options=['position', 'transparent…

## Flood Extent Mapping Using MNDWI

In [34]:
# Compute the MNDWI before flood
s2_Mndwi_pre = s2_Image_pre.normalizedDifference(["B3", "B8"]).rename("MNDWI")

In [35]:
# Visualize the MNDWI before flood
mndwi_Viz = {min:0.01, max:0.8, "palette":["#f7fbff", "#1452d9"]}

Map3 = geemap.Map(center=center, zoom = 13)
Map3.addLayer(s2_Mndwi_pre, mndwi_Viz)
Map3

Map(center=[11.875186428571428, 13.234209285714286], controls=(WidgetControl(options=['position', 'transparent…

In [37]:
# Compute the MNDWI during flood
s2_Mndwi_flood = s2_Image_flood.normalizedDifference(["B3","B8"]).rename("MNDWI")
s2_Mndwi_flood = s2_Mndwi_flood.select("MNDWI")

In [38]:
# Visualize the MNDWI after flood
Map4 = geemap.Map(center=center, zoom = 12)

Map4.addLayer(s2_Mndwi_flood, mndwi_Viz)
Map4

Map(center=[11.875186428571428, 13.234209285714286], controls=(WidgetControl(options=['position', 'transparent…

In [39]:
# Define a function that accepts an MNDWI layer and threshold value and return a mask layer
def water_mask(layer, threshold):
    mask = layer.gt(threshold).rename("water_mask").selfMask()
    return mask


In [40]:
# Extract permanent water body from the s2_mndwi_pre
perm_water = water_mask(s2_Mndwi_pre, 0)

In [41]:
# Extract flooded extent PLUS permanent water body from the s2_Mndwi_flood
flood_water = water_mask(s2_Mndwi_flood, 0)

In [42]:
# Mask out Permanent water bodies to get JUST the flood extent

# First Create a mask for the non permanent water
notPermWaterMask = s2_Mndwi_pre.lt(0).selfMask()

# Extract the flooded extent only
flooded = flood_water.updateMask(notPermWaterMask)

In [43]:
# Smooth the flooded extent layer to get a refined 
threshold = 6
connection = flooded.connectedPixelCount(26)
flooded = flooded.updateMask(connection.gt(threshold))

In [45]:
# Visualize the permanent water body overlaid on the flood layer
water_Viz = {min:0.0, max:0.9, "palette": ["#f7fbff", "#71c5f6"]}

Map5 = geemap.Map(center=center, zoom = 13)
Map5.addLayer(flooded, {min: 0.0, max:0.9, "palette": ["blue"]},"flooded")
Map5

Map(center=[11.875186428571428, 13.234209285714286], controls=(WidgetControl(options=['position', 'transparent…

In [51]:
# Convert the binary image to integer with values 0 and 1
flooded_int = flooded.multiply(255).toInt()

## Flood Damage Assessment

In [46]:
# Load and visualize the ESA land cover data
esa_LC = ee.ImageCollection('ESA/WorldCover/v200').first().clip(roi)

vis_lc = {"bands": ['Map']}

Map6 = geemap.Map(center=center, zoom = 13)
Map6.addLayer(esa_LC, vis_lc, 'Landcover')
Map6

Map(center=[11.875186428571428, 13.234209285714286], controls=(WidgetControl(options=['position', 'transparent…

In [47]:
# Create a function that export a GEE layer to GTiff 

def gee_to_tiff(input_layer, out_folder, output_filename):
    
    try:
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        file_name = os.path.join(out_folder, output_filename)
        
        # Export a GEE layer to raster Gtiff
        raster = geemap.ee_export_image(
            input_layer,
            file_name,
            scale = 10,
            region = roi,
            crs= "EPSG:32737",
            file_per_band = False
        )
        return raster
    except Exception as e:
        print("An erro has occurred:", e)

In [48]:
# Export the land cover layer as a tiff file
gee_to_tiff(esa_LC, "outputs", "landcover.tif")

Generating URL ...
Please wait ...
Data downloaded to c:\Users\HP\Downloads\Cloud\00-analytics-ready\Geospatial Information System Research and Design\projects\geospatial_analysis_flood_impacts_on_crops_in_maiduguri_nigeria\outputs\landcover.tif


In [ ]:
catalog_path = 'stac_catalog/baseline_catalog-20240910_154345/catalog.json'
catalog = pystac.Catalog.from_file(catalog_path)

print(f"Catalog ID: {catalog.id}")

In [ ]:
items = list(catalog.get_items()) 
print(f"Number of items: {len(items)}")

# Get the list of items
items = list(catalog.get_items())

# Print out the available item IDs
for item in items:
    print(item.id) 

In [ ]:


# Replace with the actual item ID you want to access
item_id = 'S2A_MSIL2A_20181117T091241_N0213_R050_T33MTR_20201129T230722.SAFE'  # Adjust as needed
baseline_stac_item = catalog.get_item(item_id)

display(baseline_stac_item)


In [ ]:
# Function to calculate NDVI
@dask.delayed
def calculate_ndvi(baseline_stac_item):

    red_href = baseline_stac_item.assets.get("BaseB04").href  # Baseline Red band
    nir_href = baseline_stac_item.assets.get("BaseB08").href  # Baseline NIR band

    if not red_href or not nir_href:
        log_progress("Red or NIR band not found in the Baseline STAC item: calculate_ndvi")
        
        client.close()
        
    else:
        try:
            log_progress(f"Processing analysis for {baseline_stac_item.id}...")

            red = rioxarray.open_rasterio(red_href, chunks="auto")
            nir = rioxarray.open_rasterio(nir_href, chunks="auto")

            # Check the number of bands
            log_progress(f"Red band number of bands: {red.shape[0]}")
            log_progress(f"NIR band number of bands: {nir.shape[0]}")
            
            # Calculate NDVI
            ndvi = (nir - red) / (nir + red)

            log_progress('Baseline NDVI calculation complete')

            # Save NDVI difference to Zarr if needed
            # now = datetime.now()
            # datetime_str = now.strftime('%Y%m%d_%H%M%S')
            group = f'kimongo-etl-job-baseline'

            ndvi.to_zarr(ZARR_STORE, mode='w', group=group)
            
            log_progress("Baseline NDVI Raster dataset saved to Zarr store.")

            return ndvi

        except Exception as e:
            log_progress(f"Error opening files: {e}")
            return None

In [1]:
# Load baseline NDVI
log_progress('Initiating Baseline NDVI calculation')

baseline_ndvi_task = calculate_ndvi(baseline_stac_item)
display(baseline_ndvi_task)


KeyboardInterrupt



In [ ]:
# Compute the NDVI task
result = dask.compute(baseline_ndvi_task)

# Access the result
baseline_ndvi = result[0]

# Optionally display the result
if baseline_ndvi is not None:
    display(baseline_ndvi)

In [ ]:
# Function to load Zarr dataset
@dask.delayed
def load_baseline_dataset(zarr_store, group):
    log_progress('Loading Baseline dataset from Zarr')
    dataset = xr.open_zarr(zarr_store, group=group, chunks='auto')
    display(dataset)
    log_progress('Loading complete')
    return dataset

In [ ]:
# Function to create NDVI map
@dask.delayed
def create_ndvi_map(ndvi_data):
    log_progress('Creating baseline NDVI Map')

    plt.figure(figsize=(10, 6))
    ndvi_plot = ndvi_data.plot(cmap='RdYlGn', vmin=-1, vmax=1)  # Using the plot method for DataArrays
    plt.title("Baseline NDVI Map")
    plt.colorbar(ndvi_plot, label='NDVI Value') 
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

    log_progress('Baseline NDVI Map created.')

In [ ]:
# Load the baseline dataset
baseline_dataset_task = load_baseline_dataset(ZARR_STORE, group="kimongo-etl-job-baseline")

# Access NDVI DataArray and create a delayed task for the NDVI map
ndvi_data_task = baseline_dataset_task['__xarray_dataarray_variable__'][0, :, :].squeeze()

# Create a task to plot the NDVI map
ndvi_map_task = create_ndvi_map(ndvi_data_task)

# Compute the tasks
dask.compute(baseline_dataset_task, ndvi_map_task)

# Close the client when done
# client.close()

In [49]:
# Create a function that converts a GEE raster layer to vector and dissolve the vector
    
def raster_to_vector(input_layer, out_folder, output_shp):

    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
        
    file_name = os.path.join(out_folder, output_shp)
    #output_file = os.path.join(out_folder, output_shp)

    # Convert raster to vector
    vector = input_layer.reduceToVectors(
        geometry= roi,
        crs= "EPSG:32737",
        scale= 10,
        geometryType='polygon',
        bestEffort=True,
        maxPixels=1e8,
        #tileScale=4
    )
    
    # convert the ee vector to shapefile and read to gdf 
    geemap.ee_to_shp(vector, file_name) 
    shp = gpd.read_file(file_name)

    # Dissolve the shapefile
    print("Attributes in the shapefile:", shp.columns) # Check for the column to dissolve
    shp_dv = shp.dissolve(by="label") # You may need to change the "label" argument depending on the attributes in your input
    shp_dv = shp_dv.to_crs("EPSG:32737")
    
    # Add Area attribute to the dissolved shapefile
    shp_dv["Area_km2"] = (shp_dv.geometry.area) / 1e6 
    print(shp_dv.crs) 
    
    # Export the shapefile
    output_shp = shp_dv.to_file(file_name, driver='ESRI Shapefile')

    return output_shp

In [52]:
# Convert the flooded extent from raster to vector
raster_to_vector(flooded_int, "outputs", "flood_dv.shp")

Attributes in the shapefile: Index(['count', 'label', 'geometry'], dtype='object')
EPSG:32737


In [53]:
# Convert the land cover from raster to vector
raster_to_vector(esa_LC, "outputs", "lc_dv.shp")

Attributes in the shapefile: Index(['count', 'label', 'geometry'], dtype='object')
EPSG:32737


In [54]:
import http.client
import time

# Write a function to determine the number of buildings affected by the flood
def building_damage(flood_shp, output_shp):

    out_folder = "Outputs"
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)

    flood_extent = os.path.join(out_folder, flood_shp)
    output_file = os.path.join(out_folder, output_shp)

    # Load the flooded shapefile
    gdf_flood = gpd.read_file(flood_extent)
    gdf_flood = gdf_flood.to_crs("EPSG:4326")
    
    # Convert it to shapely polygon
    if gdf_flood.geometry.count() > 1:
        flood_poly = gdf_flood.unary_union
    else:
        flood_poly = gdf_flood.geometry.iloc[0]

    for attempt in range(3):
        try:
            # Load osm building footprint data with the boundary of the flooded extent
            tags = {"building": True}
            bdg = ox.features_from_polygon(flood_poly, tags=tags)
            break  # Exit the loop if successful
        except http.client.ChunkedEncodingError as e:
            print(f"Error fetching OSM data: {e}. Attempting retry {attempt+1} in 5 seconds...")
            time.sleep(5)
   
    # Perform intersection to determine the buildings affected by the flood
    affected_bdg = gpd.sjoin(bdg, gdf_flood, how="inner", predicate="intersects")
    print(affected_bdg.columns)

    affected_bdg = affected_bdg.to_crs("EPSG:32737")
    # Export the shapefiles of affected buildings and select a few columns
    columns = ["geometry"] 
    affected_bdg_shp = affected_bdg[columns].to_file(output_file, driver='ESRI Shapefile')

    # Count the number of buildings
    print (f"{len(affected_bdg)} buildings were affected by the flood.")

    return affected_bdg_shp



In [55]:
# Determine the affected building
building_damage("flood_dv.shp", "affected_bdg.shp")

In [ ]:
# Land cover attribute information
lc_bands = {
    10: "Tree cover", 
    20: "Shrubland", 
    30: "Grassland", 
    40: "Cropland", 
    50: "Built-up", 
    60: "Bare / Sparse Vegetation", 
    90: "Mangroves" 
} 

In [ ]:
# Write a function to determine the exent of land cover affected by the flood
def landcover_damage(flood_shp, lc_shp, output_shp):

    out_folder = "Outputs"
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)

    flood_extent = os.path.join(out_folder, flood_shp)
    lc_extent = os.path.join(out_folder, lc_shp)
    output_file = os.path.join(out_folder, output_shp)

    # Load the flooded shapefile and the land cover shapefile
    gdf_flood = gpd.read_file(flood_extent)
    gdf_lc = gpd.read_file(lc_extent)
    
    # Perform intersection operation to determine the affected land cover
    affected_lc = gpd.overlay(gdf_flood, gdf_lc, how= "intersection", keep_geom_type=True)
    print(affected_lc.columns)
    
    # Drop and rename columns
    affected_lc.drop(columns=["label_1", "count_1", "Area_km2_1", "count_2", "Area_km2_2"], inplace = True)
    affected_lc.rename(columns={"label_2": "label",}, inplace = True) 

    # Remove permanent water bodies
    affected_lc.drop(affected_lc[affected_lc['label'] == 80].index, inplace=True)

    # Add the class column based on the label / map the name of the land covers 
    affected_lc['class'] = affected_lc['label'].map(lc_bands)

    # Re-compute the areas
    affected_lc["Area_km2"] = ((affected_lc.geometry.area) / 1e6).round(3)
    print(affected_lc.head())

    # Export the shapefiles of affected land cover
    affected_lc_shp = affected_lc.to_file(output_file, driver='ESRI Shapefile')

    return affected_lc_shp



In [ ]:
# Determine the affected land cover
landcover_damage("flood_dv.shp", "lc_dv.shp", "affected_lc_dv.shp")

## Visualize the Results of the Damage Assessment

In [ ]:
# Read the affected buildings and affected land cover files into GeodataFrames
affected_bdg = gpd.read_file("Outputs/affected_bdg.shp")
affected_lc = gpd.read_file("Outputs/affected_lc_dv.shp")

In [ ]:
# Inspect the affected land cover
affected_lc.info()
affected_lc.head(10)

In [ ]:
# Visualize the affected land cover types

# Define the color map for the classes
lc_bands_colors = {
    "Tree cover": "#006400",
    "Shrubland": "#ffbb22",
    "Grassland": "#fae500",
    "Cropland": "#ad89dd",
    "Built-up": "#e31a1c",
    "Bare / Sparse Vegetation": "#d4dadc",
    "Mangroves": "#1ec61e"
}

# Function to style each feature based on the class
def style_function(feature):
    return {
        'fillColor': lc_bands_colors.get(feature['properties']['class'], 'black'),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    }

# Initialize a geemap map
m = geemap.Map()

# Add GeoDataFrame to the map using add_gdf with style_callback
m.add_gdf(
    affected_lc,
    layer_name="Affected Land Cover",
    style={},
    hover_style={'fillOpacity': 0.8},
    style_callback=style_function,
    info_mode="on_hover",
    zoom_to_layer=True
)

# Display the map
m


In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 10))

# Visualize the affected land cover types
affected_lc.plot(
    "class", 
    legend=True,
    cmap ="Set1",
    ax=ax    
)

In [ ]:
# Create a bar chart to see the area affected for each land cover class

# List of colors for the bars based on the order of Area_km2
colors = ["#ffbb22", "#ad89dd", "#006400", "#1ec61e", "#fae500", "#e31a1c", "#d4dadc"]

# Ensure that the number of colors is equal to the number of land cover classes
if len(colors) < affected_lc.shape[0]:
    raise ValueError("Not enough colors for the number of land cover classes.")

# Sort the DataFrame by 'Area_km2' in descending order
affected_lc_sorted = affected_lc.sort_values(by='Area_km2', ascending=False)

fig, ax = plt.subplots(1, figsize=(22, 11))

# Plot the bars with different colors
ax.bar(affected_lc_sorted["class"], affected_lc_sorted["Area_km2"], color=colors[:affected_lc.shape[0]], width=0.5)

# Add labels and title
plt.xlabel("Land Cover Type")
plt.ylabel("Affected Area (km²)")
plt.title("Affected Land Cover Area by Flooding along Tana River")

In [ ]:
# Inspect the affected buildings
affected_bdg.info()
affected_bdg.head()

In [ ]:
# Visualize the affected buildings
affected_bdg.explore()

In [ ]:
# ## 2. Data Processing

# **2.1 Sentinel-2 Processing**

# Read the Sentinel-2 image using rasterio
with rasterio.open("./data/sentinel-2-l2a_20240901_1234567890_T35HKE_B04.tiff") as src:
    band_4 = src.read(1)
    meta = src.meta.copy()

# Create an xarray Dataset
ds_sentinel = xr.Dataset(
    {
        "B04": (("y", "x"), band_4),  # Band 4
    },
    coords={
        "x": ("x", src.xcoords),
        "y": ("y", src.ycoords),
    },
    attrs=meta,
)

# **2.1.1 Water Mask (Replace with your preferred method)**
# Here's an example using a simple threshold on NDWI
#  - Requires the NDWI band to be available in your Sentinel-2 data

# Calculate NDWI
ds_sentinel["NDWI"] = (ds_sentinel["B03"] - ds_sentinel["B08"]) / (
    ds_sentinel["B03"] + ds_sentinel["B08"]
)

# Threshold NDWI to create a water mask
ds_sentinel["water_mask"] = (ds_sentinel["NDWI"] < 0).astype(int)

# **2.1.2 NDVI Calculation**
ds_sentinel["NDVI"] = (ds_sentinel["B08"] - ds_sentinel["B04"]) / (
    ds_sentinel["B08"] + ds_sentinel["B04"]
)

# Set a threshold for significant NDVI change (e.g., a decrease)
ndvi_threshold = 0.5

# Create a mask for areas with significant NDVI change
ds_sentinel["ndvi_change_mask"] = (ds_sentinel["NDVI"] < ndvi_threshold).astype(int)

# Combine with the water mask (logical OR) to get a more comprehensive flood mask
ds_sentinel["flood_mask"] = (ds_sentinel["water_mask"] | ds_sentinel["ndvi_change_mask"]).astype(int)

# **2.2 OpenStreetMap Processing**
# Load the OpenStreetMap GeoDataFrame
gdf_osm = gpd.read_file("./data/maiduguri_osm.gpkg")

# Filter for relevant features (e.g., buildings, land use)
# Filter for buildings (ways and relations)
gdf_buildings = gdf_osm[
    (gdf_osm["building"] == "yes") &
    (gdf_osm["geometry"].type.isin(["Polygon", "MultiPolygon"]))
]

# Filter for land use (ways and relations)
gdf_land_use = gdf_osm[
    (gdf_osm["landuse"].isin(["forest", "farmland", "grass"])) &
    (gdf_osm["geometry"].type.isin(["Polygon", "MultiPolygon"]))
]

# Reproject to the same CRS as Sentinel-2
gdf_buildings = gdf_buildings.to_crs(ds_sentinel.crs)
gdf_land_use = gdf_land_use.to_crs(ds_sentinel.crs)



## Incorporating NDVI for Enhanced Flood Analysis

Using only the water mask for flood analysis is a simplified approach. Incorporating NDVI (Normalized Difference Vegetation Index) can significantly improve your flood analysis. Here's how to do it:

### NDVI Significance

* **Health of Vegetation:** NDVI provides an indication of the health and density of vegetation. Areas with high NDVI values generally indicate healthy vegetation.
* **Flood Impact:**  Flooded areas often experience a decrease in NDVI because of submerged vegetation or damage to crops. This change in NDVI can be a valuable indicator of flood impact.

### Incorporating NDVI into Your Flood Analysis

You can apply a threshold to the NDVI values to identify areas with significant changes in vegetation health, potentially indicating flooded areas.

In [ ]:
# ## 3. Analysis and Visualization

# **3.1 Combine Datasets**

# Create GeoDataFrames from xarray DataArrays
gdf_ndvi = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(
        [
            shapely.geometry.Polygon(
                [
                    (x, y)
                    for x, y in zip(ds_sentinel.x.values, ds_sentinel.y.values)
                ]
            )
        ]
    ),
    crs=ds_sentinel.crs,
    data={"NDVI": ds_sentinel["NDVI"].values.flatten()},
)

gdf_water_mask = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(
        [
            shapely.geometry.Polygon(
                [
                    (x, y)
                    for x, y in zip(ds_sentinel.x.values, ds_sentinel.y.values)
                ]
            )
        ]
    ),
    crs=ds_sentinel.crs,
    data={"water_mask": ds_sentinel["water_mask"].values.flatten()},
)


# Create a GeoDataFrame for the flood mask
gdf_flood_mask = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(
        [
            shapely.geometry.Polygon(
                [
                    (x, y)
                    for x, y in zip(ds_sentinel.x.values, ds_sentinel.y.values)
                ]
            )
        ]
    ),
    crs=ds_sentinel.crs,
    data={"flood_mask": ds_sentinel["flood_mask"].values.flatten()},
)

In [ ]:
import os

out_folder = "Outputs"
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

gdf_water_mask = "" # use rasterio to load from zarr store

output_file = os.path.join(out_folder, "affected_lc_disv.shp")

In [ ]:
# **3.2 Flood Impact Assessment**

# Intersect water mask with cropland layer
gdf_flooded_cropland = gpd.overlay(gdf_water_mask, gdf_land_use, how="intersection")

# Intersect water mask with building layer
gdf_flooded_buildings = gpd.overlay(gdf_water_mask, gdf_buildings, how="intersection")

# Using flood mask
gdf_flooded_cropland = gpd.overlay(gdf_flood_mask, gdf_land_use, how="intersection")

gdf_flooded_buildings = gpd.overlay(gdf_flood_mask, gdf_buildings, how="intersection")

# Alternatively:

# Perform intersection to determine the buildings affected by the flood
gdf_flooded_buildings = gpd.sjoin(gdf_buildings, gdf_water_mask, how="inner", predicate="intersects")
print(gdf_flooded_buildings.columns)

# Project the GeoDataFrame to the desired CRS (EPSG:32737 in this example)
gdf_flooded_buildings = gdf_flooded_buildings.to_crs("EPSG:32737")

# Export the shapefiles of affected buildings and select a few columns

# Select the desired columns and export as shapefile
columns = ["geometry"] 
gdf_flooded_buildings_shp = gdf_flooded_buildings[columns].to_file(output_file, driver='ESRI Shapefile')

# Count the number of buildings
print (f"{len(gdf_flooded_buildings)} buildings were affected by the flood.")

gdf_flooded_buildings_shp

# **3.3.1 Vulnerability Assessment**

# Calculate the ratio of flooded cropland to total cropland
total_cropland = gdf_land_use.area.sum()
flooded_cropland_ratio = gdf_flooded_cropland.area.sum() / total_cropland


# ... (Add population density data and calculate exposure) ...




In [ ]:
# **3.4 Visualization**

# Flood Extent Map
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
gdf_osm.plot(ax=ax, color="lightgray", edgecolor="gray")
gdf_flooded_cropland.plot(ax=ax, color="blue", alpha=0.5)
ax.set_title("Flood Extent in Maiduguri")
plt.show()

# Crop Impact Map
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
gdf_osm.plot(ax=ax, color="lightgray", edgecolor="gray")
gdf_ndvi.plot(ax=ax, column="NDVI", cmap="viridis", legend=True)
ax.set_title("Crop Impact Assessment (NDVI)")
plt.show()

# Vulnerability Map (example using a heatmap)
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
gdf_osm.plot(ax=ax, color="lightgray", edgecolor="gray")
gdf_flooded_cropland.plot(ax=ax, column="water_mask", cmap="Reds", legend=True)
ax.set_title("Vulnerability Assessment (Flooded Cropland)")
plt.show()

# **3.5 3D Visualization (Example)**

# Create a PyVista grid
grid = pv.StructuredGrid(ds_sentinel.x.values, ds_sentinel.y.values, ds_sentinel.NDVI.values)

# Plot the NDVI data
pl = pv.Plotter()
pl.add_mesh(grid, scalars="NDVI", cmap="viridis")
pl.show()